# Busca (Parte I)

## Mochila com Número Irrestrito de Itens com Valor

In [1]:
max_size = 19
items = [(1,3),(4,6),(5,7)]

Tamanho de Estados

In [2]:
def state_size(state, items):
    size = 0
    for i in range(len(state)):
        size += state[i] * items[i][1]
    return size

In [3]:
state_size([1,1,1], items)

16

Avaliação de Estados

In [4]:
def evaluate_state(state, items):
    value = 0
    for i in range(len(state)):
        value += state[i] * items[i][0]
    return value

In [5]:
evaluate_state([1,2,0], items)

9

Geração de Estados

In [6]:
def generate_states(initial_state):
    states = []
    for i in range (len(initial_state)):
        aux = initial_state.copy()
        aux[i] = initial_state[i] + 1
        states.append(aux)
    return states

In [7]:
generate_states([0,0,0])

[[1, 0, 0], [0, 1, 0], [0, 0, 1]]

Métodos Heurísticos - Construtivos

Hill Climbing Determinístico

In [8]:
import time

def hill_climbing(max_size, items, max_time):
    start = time.process_time()
    current_state = [0]*len(items)
    optimal_value = 0
    optimal_size = 0
    optimal_state = current_state
    valid_states = len(items)
    end = 0

    while valid_states != 0 and end-start <= max_time:
        valid_states = len(items)
        possible_states = generate_states(current_state)

        for state in possible_states:
            aux_val = evaluate_state(state, items)
            aux_size = state_size(state, items)
            if aux_size <= max_size:
                if aux_val > optimal_value:
                    optimal_value = aux_val
                    optimal_size = aux_size
                    optimal_state = state
                    current_state = state
            else:
                valid_states = valid_states - 1

        end = time.process_time()

    return optimal_state, optimal_size, optimal_value

In [9]:
hill_climbing(max_size, items, 120)

([1, 0, 2], 17, 11)

Roleta

Soma de valores de todos os estados

In [10]:
def states_total_value(states):
    total_sum = 0
    for state in states:
        total_sum = total_sum + state[0]
    return total_sum

In [11]:
states_total_value([(1, [1,0,0]), (4, [0, 1, 0]), (5, [0, 0, 1])])

10

Construção da roleta

In [12]:
def roulette_construction(states):
    aux_states = []
    roulette = []
    total_value = states_total_value(states)

    for state in states:
        value = state[0]
        if total_value != 0:
            ratio = value/total_value
        else:
            ratio = 1
        aux_states.append((ratio,state[1]))
 
    acc_value = 0
    for state in aux_states:
        acc_value = acc_value + state[0]
        s = (acc_value,state[1])
        roulette.append(s)
    return roulette

In [13]:
roulette_construction([(1, [1,0,0]), (4, [0, 1, 0]), (5, [0, 0, 1])])

[(0.1, [1, 0, 0]), (0.5, [0, 1, 0]), (1.0, [0, 0, 1])]

Rodar a roleta

In [14]:
import random

def roulette_run (rounds, roulette):
    if roulette == []:
        return []
    selected = []
    while len(selected) < rounds:
        r = random.uniform(0,1)
        for state in roulette:
            if r <= state[0]:
                selected.append(state[1])
                break
    return selected

In [15]:
roulette_run (1, [(0.1, [1, 0, 0]), (0.5, [0, 1, 0]), (1.0, [0, 0, 1])])

[[0, 1, 0]]

Hill Climbing Não Determinístico

In [16]:
import time

def hill_climbing_nd(max_size, items, max_time):
    start = time.process_time()
    current_state = [0]*len(items)
    optimal_value = 0
    optimal_size = 0
    optimal_state = current_state
    valid_states = len(items)
    end = 0

    while valid_states != 0 and end-start <= max_time:
        valid_states = len(items)
        possible_states = generate_states(current_state)
        
        states = []
        for state in possible_states:
            aux_val = evaluate_state(state, items)
            aux_size = state_size(state, items)
            if aux_size <= max_size:
                states.append((aux_val, state))
                if aux_val > optimal_value:
                    optimal_value = aux_val
                    optimal_size = aux_size
                    optimal_state = state
            else:
                valid_states = valid_states - 1                
        print (states)
        
        if valid_states > 0:
            current_state = roulette_run (1, roulette_construction(states))[0]
            print (current_state)

        end = time.process_time()

    return optimal_state, optimal_size, optimal_value

In [17]:
hill_climbing_nd(max_size, items, 120)

[(1, [1, 0, 0]), (4, [0, 1, 0]), (5, [0, 0, 1])]
[0, 0, 1]
[(6, [1, 0, 1]), (9, [0, 1, 1]), (10, [0, 0, 2])]
[0, 1, 1]
[(10, [1, 1, 1]), (13, [0, 2, 1])]
[0, 2, 1]
[]


([0, 2, 1], 19, 13)

Beam Search Determinístico

In [18]:
import time

def return_value(state_tuple):
    return state_tuple[0]

def beam_search(max_size, items, tam_viga, max_time):
    start = time.process_time()
    current_state = [0]*len(items)
    queue = [(evaluate_state(current_state,items),current_state)]
    end = 0
    
    while queue and end-start <= max_time:
        states = []
        for state in queue:
            possible_states = generate_states(state[1])
            for state in possible_states:
                if state_size(state, items) <= max_size:
                    states.append((evaluate_state(state,items),state))
        if states == []:
            break
        else:
            states.sort(key = return_value,reverse = True)
            queue = states[:tam_viga]

        if len(states) > 0:
            current_state = roulette_run (1, roulette_construction(states))[0]
            print (current_state)

        end = time.process_time()    

    return queue[0][1], state_size(queue[0][1], items), queue[0][0]

In [19]:
beam_search(max_size, items, 2, 120)

[1, 0, 0]
[0, 0, 2]
[1, 1, 1]


([0, 2, 1], 19, 13)

### Exercícios de Fixação

1. Caracterize os tipos de problemas de busca nos quais é interessante usar métodos heurísticos.

É interessante o uso da heuristica quando é necessário a busca de algum resultado de forma rápida e eficiente, não necessitando de ser perfeito, mas no minimo ótimo. Nisso, alguns problemas são mais relevantes, como organização de uma mochila, caxeiro viajante, ordenação de peças de marmore, etc.

2.  Era uma vez um fazendeiro que foi ao mercado e comprou um lobo, um carneiro e uma alface. No caminho para casa, o fazendeiro chegou à margem de um rio e arrendou um barco. Mas, na travessia do rio por barco, o agricultor poderia levar apenas a si mesmo e uma única de suas compras - o lobo, o carneiro, ou a alface. Se fossem deixados sozinhos em uma mesma margem, o lobo comeria o carneiro e o carneiro comeria a alface. O desafio do fazendeiro é atravessar a si mesmo e as suas compras para a margem oposta do rio, deixando cada compra intacta. Crie uma representação para descrever os estados desse problema. Partindo de um estado inicial no qual fazendeiro, lobo, carneiro e alface se encontram na mesma margem do rio, desenhe uma árvore com o espaço dos possíveis estados que podem ser gerados a partir daí. Descarte continuar ramos da árvores a partir de estados inválidos e não possibilite em uma mesma ramificação o retorno a um estado previamente visitado. Apresente as soluções encontradas.  

2. Diferencie heurísticas de metaheurísticas. Contraste vantagens de cada uma.

A principal diferencia da heuristica para a metaheuristica é que a metaheuristica é para solucionar um problema generico, já a heuristica não. 

Metaheuristica: Foge de ótimos locais, não é deterministico e normalmente fácil para adaptar a novos problemas. 
Heuristica: Espeficio a um problema, normalmente baseados na experiencia e pode ser desenvolvido qualquer artificio para melhorar a qualidade da execução do código e da resposta. O problema é que pode ser extremamente trabalhoso

2. Porque métodos determinísticos de busca heurística tendem a não alcançar bons resultados em resolução de problemas não lineares?

Como são aproximados a uma busca cega, necessitam de muito tempo para produzir alguma solução, ou até nenhuma pois necessitam de muito poder de processamento.

3. Implementar o método beam search não determinístico.

In [20]:
import time

def return_value(state_tuple):
    return state_tuple[0]

def beam_search(max_size, items, tam_viga, max_time):
    start = time.process_time()
    current_state = [0]*len(items)
    queue = [(evaluate_state(current_state,items),current_state)]
    end = 0
    
    while queue and end-start <= max_time:
        states = []
        for state in queue:
            possible_states = generate_states(state[1])
            for state in possible_states:
                if state_size(state, items) <= max_size:
                    states.append((evaluate_state(state,items),state))
        if states == []:
            break
        else:
            states.sort(key = return_value,reverse = True)
            queue = states[:tam_viga]

        end = time.process_time()    

    return queue[0][1], state_size(queue[0][1], items), queue[0][0]

4. Alterar o algoritmo de hill climbing determinístico para levar em consideração a existência de um número finito de elementos de cada item. Cada item tem um número finito e possivelmente diferente de elementos disponíveis.

In [21]:
import time

def hill_climbing(max_size, items, max_time):
    start = time.process_time()
    current_state = [0]*len(items)
    optimal_value = 0
    optimal_size = 0
    optimal_state = current_state
    valid_states = len(items)
    end = 0

    while valid_states != 0 and end-start <= max_time:
        valid_states = len(items)
        possible_states = generate_states(current_state)

        for state in possible_states:
            aux_val = evaluate_state(state, items)
            aux_size = state_size(state, items)
            if aux_size <= max_size:
                if aux_val > optimal_value:
                    optimal_value = aux_val
                    optimal_size = aux_size
                    optimal_state = state
                    current_state = state
            else:
                valid_states = valid_states - 1

        end = time.process_time()

    return optimal_state, optimal_size, optimal_value